In [1]:
## This script is for data preparation

In [2]:
# step 1. load packages
import os, cv2, h5py, gdal
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [3]:
# step 2. data preparation
## load tif images
imTIF = gdal.Open('../training/20170305_PRday4-92-57.tif')
## get three color channels
channel_1 = np.array(imTIF.GetRasterBand(1).ReadAsArray())
channel_2 = np.array(imTIF.GetRasterBand(2).ReadAsArray())
channel_3 = np.array(imTIF.GetRasterBand(3).ReadAsArray())
## transform to .jpg
img = np.array([channel_3, channel_2, channel_1])
img = np.transpose(img, [1, 2, 0])

In [4]:
# step 3. transform images
imgH = img
## down scale the image
imgL = cv2.resize(img, (1666, 1666), interpolation = cv2.INTER_NEAREST)
## upscale image using cubic spline
imgL = cv2.resize(imgL, (10000, 10000), interpolation = cv2.INTER_CUBIC)
## YCrCb transformation
#imgH = cv2.cvtColor(imgH, cv2.COLOR_BGR2YCrCb)[:, :, 0] / 255.0
#imgL = cv2.cvtColor(imgL, cv2.COLOR_BGR2YCrCb)[:, :, 0] / 255.0
imgH = imgH / 255.0
imgL = imgL / 255.0

In [5]:
# step 4. save image patch as h5py
def write_hdf5(x, y, output_filename):
    with h5py.File(output_filename, 'w') as h:
        h.create_dataset('data', data=x, shape=x.shape)
        h.create_dataset('label', data=y, shape=y.shape)
    print('saving completed')
X = []
Y = []
## split image into 200 x 200 patches
for i in range(50):
    for j in range(50):
        r_1 = i * 200
        r_2 = r_1 + 200
        c_1 = j * 200
        c_2 = c_1 + 200
        X.append(imgL[r_1:r_2, c_1:c_2])
        Y.append(imgH[r_1:r_2, c_1:c_2])
X = np.array(X, dtype=np.float32)
Y = np.array(Y, dtype=np.float32)
write_hdf5(X, Y, 'crop_train.h5')      

saving completed
